In [45]:
%%capture
%run Baseball_Project_Part_3.ipynb

In [48]:
from sklearn.utils import resample
pared_full_data['WSWin'] = WSWin


In [104]:
prob_df = pd.DataFrame()
sc = StandardScaler()
pca = PCA(n_components = .90, svd_solver = 'full', random_state = 42)
enn = EditedNearestNeighbours(kind_sel = 'all', random_state = 42, n_neighbors = 25)
lr = LogisticRegression(class_weight = 'balanced', solver = 'liblinear', C = 2.701, random_state = 100)
pipe = make_pipeline(sc, pca, enn, lr)

for i in range(1000):
    
    boot_data = resample(pared_full_data, n_samples = 540, stratify = WSWin)
    boot_WSWin = boot_data.WSWin
    boot_data.drop(['WSWin'], inplace = True, axis = 1)
    pipe.fit(boot_data, boot_WSWin);
    
    WS_prob = pipe.predict_proba(pared_data_2018)[:,1]
    prob_df = pd.concat([prob_df, pd.DataFrame(WS_prob[np.newaxis, :], columns = Teams)])

In [101]:
#pipe.fit(boot_data, boot_WSWin);

In [105]:
#prob_df = pd.DataFrame()

#WS_prob = pipe.predict_proba(pared_data_2018)[:,1]
#prob_df = pd.concat([prob_df, pd.DataFrame(WS_prob[np.newaxis, :], columns = Teams)])
prob_df

Team,Red Sox,Yankees,Dodgers,Indians,Athletics,Astros,Nationals,Rockies,Rays,Brewers,...,Phillies,Diamondbacks,Mets,White Sox,Royals,Orioles,Tigers,Padres,Giants,Marlins
0,0.996326,0.215712,0.296468,0.800734,0.497078,0.001417,0.293109,0.001585,0.174795,0.007693,...,8.275006e-07,3.017760e-05,0.000207,7.648514e-09,2.249579e-04,3.602343e-10,1.419434e-07,1.493485e-09,2.531684e-06,1.634435e-07
0,0.995138,0.003363,0.321933,0.993061,0.994832,0.001671,0.003271,0.000067,0.011358,0.014874,...,4.794547e-09,6.014877e-07,0.000030,1.107954e-07,8.594248e-02,4.508484e-09,3.825882e-09,4.365078e-09,1.407535e-07,5.478778e-06
0,0.730200,0.142446,0.759693,0.941607,0.231533,0.006831,0.115543,0.160366,0.365707,0.264819,...,7.960248e-08,3.254252e-04,0.000699,8.579145e-08,1.597221e-06,1.164163e-11,4.126023e-09,1.749922e-09,1.474938e-07,6.936320e-08
0,0.284711,0.098801,0.990325,0.091082,0.339139,0.029845,0.280020,0.130948,0.000553,0.401588,...,1.235635e-05,2.601182e-02,0.005864,1.552793e-08,2.049495e-07,2.797682e-10,5.613031e-07,2.664846e-06,8.782098e-06,2.881317e-05
0,0.976417,0.028291,0.035151,0.695579,0.008484,0.003139,0.120218,0.003522,0.110350,0.000698,...,4.567614e-07,2.440684e-05,0.000074,4.496395e-08,6.686358e-05,2.121171e-11,3.974786e-09,7.704821e-10,6.529548e-07,4.412737e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0.992052,0.095133,0.005120,0.915153,0.094633,0.002850,0.241418,0.005899,0.675979,0.001188,...,7.997750e-08,3.609842e-05,0.000097,6.288332e-08,4.904781e-02,1.227689e-10,1.320066e-06,6.628527e-10,6.823639e-06,2.691673e-07
0,0.999421,0.162289,0.231352,0.071163,0.031396,0.000834,0.012452,0.000255,0.066003,0.018351,...,2.570592e-05,3.107922e-04,0.001136,1.493163e-07,8.531063e-07,2.852219e-10,7.164660e-07,1.106082e-09,4.558396e-07,1.571233e-05
0,0.996376,0.027890,0.046719,0.998624,0.971106,0.000969,0.414654,0.007446,0.073217,0.088054,...,1.145559e-07,2.224404e-03,0.000111,4.228272e-08,2.770254e-02,3.901698e-11,1.003483e-05,1.258424e-09,2.491197e-03,3.174533e-05
0,0.997789,0.021822,0.099283,0.079485,0.004429,0.002282,0.297046,0.051212,0.035814,0.005026,...,2.201856e-05,2.140187e-03,0.000028,1.369581e-09,9.686065e-07,2.965548e-13,1.348687e-09,4.790456e-10,8.515925e-08,2.493496e-08


In [103]:
a = pd.DataFrame([[1,4,5]], columns = ['A', 'B', 'C'])
b = pd.DataFrame([[6,8,3]], columns = ['C', 'A', 'B'])
display(a)
display(b)

,A,B,C
0,1,4,5


,C,A,B
0,6,8,3


In [73]:
a.merge(b, how = 'outer')

,A,B,C
0,1,4,5
1,8,3,6
